### Create a matrix of size num_models x num_folds containing normalized loglikelihood for both train and test splits

In [1]:
"""
Imports
"""
import json
import sys
import numpy as np
import os

from post_processing_utils import load_data, load_session_fold_lookup, \
    prepare_data_for_cv, calculate_baseline_test_ll, \
    calculate_glm_test_loglikelihood, calculate_cv_bit_trial, \
    return_glmhmm_nll, return_lapse_nll

In [2]:
"""
Parameters
"""
# Parameters
C = 2  # number of output classes
num_folds = 5  # number of folds
D = 1  # number of output dimensions
K_max = 5  # maximum number of latent states
num_models = K_max + 2  # model for each latent + 2 lapse models
bin_num = 10

In [4]:
"""
DIRECTORIES
"""
data_dir = '../../data/ibl/data_for_cluster/'
data_dir = '/home/ines/repositories/representation_learning_variability/DATA/GLMHMM/'
data_dir = '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/GLMHMM/'

"""
Create folders to save processed data
"""
# Create directories for saving data:
processed_ibl_data_path = data_dir + "data_for_cluster/" + "data_by_bin" + str(bin_num) + "global_normalization/"

# Create directory for results:
results_dir = '../../results/ibl_global_fit/'


In [5]:

# Load data
inpt, y, bin_data = load_data(processed_ibl_data_path + 'all_animals_concat.npz')
bin_fold_lookup_table = load_session_fold_lookup(
    processed_ibl_data_path + 'all_animals_concat_session_fold_lookup.npz')

# For later saving the data
animal_preferred_model_dict = {}
models = ["GLM", "Lapse_Model", "GLM_HMM"]

cvbt_folds_model = np.zeros((num_models, num_folds))
cvbt_train_folds_model = np.zeros((num_models, num_folds))

# Save best initialization for each model-fold combination
best_init_cvbt_dict = {}
for fold in range(num_folds):
    test_inpt, test_y, test_nonviolation_mask, this_test_session, \
    train_inpt, train_y, train_nonviolation_mask, this_train_session, M,\
    n_test, n_train = prepare_data_for_cv(
        inpt, y, bin_data, bin_fold_lookup_table, fold)
    
    
    # Ines addapted
    ll0 = calculate_baseline_test_ll(
        train_y[0, train_nonviolation_mask == 1],
        test_y[0, test_nonviolation_mask == 1], C)
    ll0_train = calculate_baseline_test_ll(
        train_y[0, train_nonviolation_mask == 1],
        train_y[0, train_nonviolation_mask == 1], C)

    for model in models:
        print("model = " + str(model))
        if model == "GLM":
            # Load parameters and instantiate a new GLM object with
            # these parameters
            glm_weights_file = results_dir + '/GLM/fold_' + str(
                fold) + '/variables_of_interest_iter_0.npz'
            
            # Ines addapted, because data is in a different format
            ll_glm = calculate_glm_test_loglikelihood(
                glm_weights_file, test_y[0, test_nonviolation_mask == 1],
                test_inpt[0, test_nonviolation_mask == 1], M, C)
            ll_glm_train = calculate_glm_test_loglikelihood(
                glm_weights_file, train_y[0, train_nonviolation_mask == 1],
                train_inpt[0, train_nonviolation_mask == 1], M, C)
            
            cvbt_folds_model[0, fold] = calculate_cv_bit_trial(
                ll_glm, ll0, n_test)
            cvbt_train_folds_model[0, fold] = calculate_cv_bit_trial(
                ll_glm_train, ll0_train, n_train)
            
# Save cvbt_folds_model as numpy array for easy parsing across all
# models and folds
np.savez(results_dir + "/cvbt_folds_model.npz", cvbt_folds_model)
np.savez(results_dir + "/cvbt_train_folds_model.npz",
            cvbt_train_folds_model)


model = GLM


FileNotFoundError: [Errno 2] No such file or directory: '../../results/ibl_global_fit//GLM/fold_0/variables_of_interest_iter_0.npz'